<a href="https://colab.research.google.com/github/newmantic/MVO/blob/main/MVO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from scipy.optimize import minimize

def calculate_portfolio_return(weights: np.ndarray, returns: np.ndarray) -> float:
    """
    Calculate the expected return of the portfolio.

    Args:
    weights (np.ndarray): Array of portfolio weights.
    returns (np.ndarray): Array of expected returns for each asset.

    Returns:
    float: Expected portfolio return.
    """
    return np.dot(weights, returns)

def calculate_portfolio_variance(weights: np.ndarray, covariance_matrix: np.ndarray) -> float:
    """
    Calculate the variance of the portfolio.

    Args:
    weights (np.ndarray): Array of portfolio weights.
    covariance_matrix (np.ndarray): Covariance matrix of asset returns.

    Returns:
    float: Portfolio variance.
    """
    return np.dot(weights.T, np.dot(covariance_matrix, weights))

def calculate_sharpe_ratio(weights: np.ndarray, returns: np.ndarray, covariance_matrix: np.ndarray, risk_free_rate: float) -> float:
    """
    Calculate the Sharpe Ratio of the portfolio.

    Args:
    weights (np.ndarray): Array of portfolio weights.
    returns (np.ndarray): Array of expected returns for each asset.
    covariance_matrix (np.ndarray): Covariance matrix of asset returns.
    risk_free_rate (float): Risk-free rate.

    Returns:
    float: Sharpe Ratio of the portfolio.
    """
    portfolio_return = calculate_portfolio_return(weights, returns)
    portfolio_variance = calculate_portfolio_variance(weights, covariance_matrix)
    portfolio_std_dev = np.sqrt(portfolio_variance)
    return (portfolio_return - risk_free_rate) / portfolio_std_dev

def optimize_portfolio_weights(returns: np.ndarray, covariance_matrix: np.ndarray, risk_free_rate: float, target_return: float = None) -> np.ndarray:
    """
    Optimize the portfolio weights to maximize the Sharpe Ratio or minimize variance for a given target return.

    Args:
    returns (np.ndarray): Array of expected returns for each asset.
    covariance_matrix (np.ndarray): Covariance matrix of asset returns.
    risk_free_rate (float): Risk-free rate.
    target_return (float): Target portfolio return (if None, optimize for maximum Sharpe Ratio).

    Returns:
    np.ndarray: Optimized portfolio weights.
    """
    num_assets = len(returns)

    def portfolio_variance(weights):
        return calculate_portfolio_variance(weights, covariance_matrix)

    def constraint_sum_weights(weights):
        return np.sum(weights) - 1.0

    if target_return is not None:
        def constraint_target_return(weights):
            return calculate_portfolio_return(weights, returns) - target_return
        constraints = [{'type': 'eq', 'fun': constraint_sum_weights},
                       {'type': 'eq', 'fun': constraint_target_return}]
    else:
        def negative_sharpe_ratio(weights):
            return -calculate_sharpe_ratio(weights, returns, covariance_matrix, risk_free_rate)
        constraints = [{'type': 'eq', 'fun': constraint_sum_weights}]

    bounds = [(0.0, 1.0) for _ in range(num_assets)]
    initial_weights = np.ones(num_assets) / num_assets

    if target_return is not None:
        result = minimize(portfolio_variance, initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)
    else:
        result = minimize(negative_sharpe_ratio, initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)

    return result.x

In [2]:
# Test Data
expected_returns = np.array([0.05, 0.07, 0.06])  # Expected returns for each asset
covariance_matrix = np.array([[0.001, 0.0008, 0.0002],
                              [0.0008, 0.002, 0.0004],
                              [0.0002, 0.0004, 0.001]])  # Covariance matrix of returns
risk_free_rate = 0.02

# Test case 1: Calculate Portfolio Return
weights = np.array([0.4, 0.4, 0.2])
portfolio_return = calculate_portfolio_return(weights, expected_returns)
print(f"Portfolio Return: {portfolio_return:.4f}")  # Expected: Weighted sum of returns

# Test case 2: Calculate Portfolio Variance
portfolio_variance = calculate_portfolio_variance(weights, covariance_matrix)
print(f"Portfolio Variance: {portfolio_variance:.6f}")  # Expected: Variance of portfolio

# Test case 3: Calculate Sharpe Ratio
sharpe_ratio = calculate_sharpe_ratio(weights, expected_returns, covariance_matrix, risk_free_rate)
print(f"Sharpe Ratio: {sharpe_ratio:.4f}")  # Expected: Sharpe Ratio of portfolio

# Test case 4: Optimize Portfolio Weights for Maximum Sharpe Ratio
optimal_weights = optimize_portfolio_weights(expected_returns, covariance_matrix, risk_free_rate)
print(f"Optimized Weights for Maximum Sharpe Ratio: {optimal_weights}")  # Expected: Array of optimal weights

# Test case 5: Optimize Portfolio Weights for Target Return
target_return = 0.065
optimal_weights_target_return = optimize_portfolio_weights(expected_returns, covariance_matrix, risk_free_rate, target_return)
print(f"Optimized Weights for Target Return {target_return}: {optimal_weights_target_return}")  # Expected: Array of optimal weights for target return

Portfolio Return: 0.0600
Portfolio Variance: 0.000872
Sharpe Ratio: 1.3546
Optimized Weights for Maximum Sharpe Ratio: [0.21951693 0.23072551 0.54975756]
Optimized Weights for Target Return 0.065: [0.08311111 0.58311112 0.33377778]
